In [ ]:
import gym
from gym import spaces
import serial
import time
from tqdm.notebook import tqdm
import csv
from datetime import datetime
import matplotlib.pyplot as plt

import datetime 

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C, PPO2
import numpy as np


import serial
import csv

import warnings
warnings.filterwarnings("ignore")

/home/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [ ]:
class handGoalEnv(gym.Env):
    """A goal-based environment. It functions just as any regular OpenAI Gym environment but it
    imposes a required structure on the observation_space. More concretely, the observation
    space is required to contain at least three elements, namely `observation`, `desired_goal`, and
    `achieved_goal`. Here, `desired_goal` specifies the goal that the agent should attempt to achieve.
    `achieved_goal` is the goal that it currently achieved instead. `observation` contains the
    actual observations of the environment as per usual.
    """
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self, arduino_port_motor, arduino_port_sensor, time_observation = 1000, threshold = 5000):
        super(handGoalEnv, self).__init__()
             
        self.arduino_port_motor = arduino_port_motor
        self.arduino_port_sensor = arduino_port_sensor

        self.time_observation = time_observation

        self.action_space = spaces.Discrete(511)
        self.observation_space = spaces.Box(low=-10e+4, high=10e+4, 
                                             shape=(3,time_observation), 
                                             dtype=np.float16)
    
        #self.goal_obs = 200#self.compute_noise() #level of noise
        self.threshold = threshold #the threshold after which a goal is considered achieved
        
        self.number_of_episodes = 0
        self.done_moment = 0
        
        self.sped = 250
        
    '''   
    def compute_noise(self):
        self.motor(0)
        noise = self._next_observation()
        noise_lvl = self.compute_reward(noise)
        print('noise lvl:', noise_lvl)
        return noise_lvl
    '''     
    def reset(self):
        
        # Reset the state of the environment to an initial state
        print('Reset')
        
        print('done_moment:', self.done_moment)
        
        self.done_moment = 0
        
#         if self.done_max < self.done_moment:
#             self.done_max = self.done_moment
        
        speed = 250
        self.motor(speed, feedback = True)
        
        return self.sensors(rse = True)
        
        # Enforce that each GoalEnv uses a Goal-compatible observation space.
#         if not isinstance(self.observation_space, gym.spaces.Dict):
#             raise error.Error('GoalEnv requires an observation space of type gym.spaces.Dict')
#         for key in ['observation', 'achieved_goal', 'desired_goal']:
#             if key not in self.observation_space.spaces:
#                 raise error.Error('GoalEnv requires the "{}" key to be part of the observation dictionary.'.format(key))

    def compute_reward(self, obs):#achieved_goal, desired_goal, info):
        """Compute the step reward. This externalizes the reward function and makes
        it dependent on an a desired goal and the one that was achieved. If you wish to include
        additional rewards that are independent of the goal, you can include the necessary values
        to derive it in info and compute it accordingly.
        Args:
            achieved_goal (object): the goal that was achieved during execution
            desired_goal (object): the desired goal that we asked the agent to attempt to achieve
            info (dict): an info dictionary with additional information
        Returns:
            float: The reward that corresponds to the provided achieved goal w.r.t. to the desired
            goal. Note that the following should always hold true:
                ob, reward, done, info = env.step()
                assert reward == env.compute_reward(ob['achieved_goal'], ob['goal'], info)
        """
        vector_mean_x = np.full( len(obs[0])  ,np.mean(obs[0]))
        vector_mean_y = np.full( len(obs[1])  ,np.mean(obs[1]))
        vector_mean_z = np.full( len(obs[2])  ,np.mean(obs[2]))
        
        x_new = np.sum((np.subtract(obs[0], vector_mean_x))**2)
        y_new = np.sum((np.subtract(obs[1], vector_mean_y))**2)
        z_new = np.sum((np.subtract(obs[2], vector_mean_z))**2)
        
        
        return - np.sum([x_new, y_new, z_new])/3
#         # Compute distance between goal and the achieved goal.
#         d = goal_distance(achieved_goal, goal) #np.linalg.norm(achieved_goal - goal, axis=-1)
#         return -d
        
    def step(self, action):
        # Execute one time step within the environment
        """Run one timestep of the environment's dynamics. When end of
        episode is reached, you are responsible for calling `reset()`
        to reset this environment's state.
        Accepts an action and returns a tuple (observation, reward, done, info).
        Args:
            action (object): an action provided by the agent
        Returns:
            observation (object): agent's observation of the current environment
            reward (float) : amount of reward returned after previous action
            done (bool): whether the episode has ended, in which case further step() calls will return undefined results
            info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        self.number_of_episodes += 1
        
        if self.number_of_episodes%10 == 0:
            print('==================================================')
            print('Episode:', self.number_of_episodes)
            self.motor(speed = 0)
            time.sleep(50)
#             self.goal_obs = self.compute_noise()
#             time.sleep(20)
            print('==================================================')
        
        print('____________________________________________')
        print('action =', action)
        self.sped = action
        
        self.motor(action)
        time.sleep(10)
        
        obs = self._next_observation()
        
        rew = self.compute_reward(obs)
        
        distance = abs(rew) #- self.goal_obs) 
        
        done = bool(distance < self.threshold)
        
        print('reward:', rew)
        print('distance:', distance, 'done:', done)
        
        reward = rew
        
        if not done:
            self.done_moment += 1
        
        return obs, reward, done, {}
    
    
    def motor(self, speed, feedback = True):
        self.arduino_port_motor.write(str(speed).encode())
        if feedback == True:
            print ("RECIEVED BACK:",self.arduino_port_motor.readline().decode())
    
    
    def sensors(self, rse = False):
        
        if rse == False:
        
            column_names = ['x', 'y', 'z']
            file_name =  '9v' +'_' + str(self.sped) + '.csv'
            file_ = open(file_name, "w")
            writer = csv.writer(file_, delimiter = ",")
            writer.writerow(column_names)
        else:
            print('reset')
        
        self.arduino_port_sensor.reset_input_buffer()
        self.arduino_port_sensor.reset_output_buffer()
        self.arduino_port_sensor.write(str(1).encode())
        #print(arduino_port_sensor.readline())
        command = self.arduino_port_sensor.readline().decode().rstrip().replace(" ", "")
        print('comand:', command)
        main_array = [command]
        #pbar = tqdm(total=self.time_observation)
        #self.time_start = str(datetime.now())[14:-3]
        while True:
            values = self.arduino_port_sensor.readline().decode().rstrip().replace(" ", "")
            #print('values:', values)
            main_array.append(values)
            #pbar.update(1)
            if values == '':
                print('!!!!')
                continue
                #self.sensors()
            if values == '0':
                #self.arduino_port_sensor.write(str(0).encode())
                break
       
        #self.time_end = str(datetime.now())[14:-3]
        
        #pbar.close()
        #print('len:', len(main_array))
        main_array = main_array[1:len(main_array)-1]
        x = []
        y = []
        z = []
        #print(main_array)   
        for each_str in main_array:
            result = [x.strip() for x in each_str.split(',')]
            #print(result)
            x.append(float(result[0]))
            y.append(float(result[1]))
            z.append(float(result[2]))
            if rse == False:
                writer.writerow([float(result[0]), float(result[1]), float(result[2])])
            
        if rse == False:
            file_.close()
        #print('ok')
            
        return [x, y, z]

    
    def _next_observation(self):
        return self.sensors()

In [ ]:
arduino_port_motor = serial.Serial('/dev/ttyACM4', 9600)
arduino_port_motor = serial.Serial('/dev/ttyACM4', 9600)
arduino_port_sensor = serial.Serial('/dev/ttyACM0', 9600, 
                                    timeout = 1)

env = handGoalEnv(arduino_port_motor , arduino_port_sensor)

#model = PPO2(MlpPolicy, env, tensorboard_log="./loser_tensorboard/")

In [ ]:
model.learn(total_timesteps=500, tb_log_name="7.5v")

Reset
done_moment: 0
RECIEVED BACK: 250

reset
comand: 1
____________________________________________
action = 360
RECIEVED BACK: 360

comand: 1
reward: -23543.887
distance: 23543.887 done: False
____________________________________________
action = 49
RECIEVED BACK: 49

comand: 1
reward: -25621.47566666667
distance: 25621.47566666667 done: False
____________________________________________
action = 311
RECIEVED BACK: 311

comand: 1
reward: -26747.150666666672
distance: 26747.150666666672 done: False
____________________________________________
action = 430
RECIEVED BACK: 430

comand: 1
reward: -147615.12600000002
distance: 147615.12600000002 done: False
____________________________________________
action = 16
RECIEVED BACK: 16

comand: 1
reward: -22578.862000000005
distance: 22578.862000000005 done: False
____________________________________________
action = 324
RECIEVED BACK: 324

comand: 1
reward: -22986.79533333333
distance: 22986.79533333333 done: False
___________________________

KeyboardInterrupt: 

In [ ]:
model.learn(total_timesteps=500, tb_log_name="4.5v", reset_num_timesteps=False)

Reset
done_moment: 0


KeyboardInterrupt: 

In [ ]:
model.learn(total_timesteps=500, tb_log_name="train2")

Reset
done_moment: 5
RECIEVED BACK: 250

comand: 1
____________________________________________
action = 32
RECIEVED BACK: 32

comand: 1
reward: -837.5969999999999
distance: 1037.5969999999998 done: False
____________________________________________
action = 485
RECIEVED BACK: 485

comand: 1
reward: -60185.543000000005
distance: 60385.543000000005 done: False
____________________________________________
action = 162
RECIEVED BACK: 162

comand: 1
reward: -67730.11933333332
distance: 67930.11933333332 done: False
____________________________________________
action = 441
RECIEVED BACK: 441

comand: 1
reward: -60694.90533333333
distance: 60894.90533333333 done: False
____________________________________________
action = 189
RECIEVED BACK: 189

comand: 1
reward: -65771.93933333333
distance: 65971.93933333333 done: False
____________________________________________
action = 354
RECIEVED BACK: 354

comand: 1
reward: -6340.630333333332
distance: 6540.630333333332 done: False
__________________

KeyboardInterrupt: 

In [ ]:
arduino_port_motor.write(str(0).encode())
print("RECIEVED BACK:",arduino_port_motor.readline().decode())

RECIEVED BACK: 0

